# [LVV-T2732] - StarTracker Pointing and Tracking Test - Slew and Settle - TMA Tracking Jitter Validation

**Requirements:**
 - All the MT components should be enabled.
 - Need the `notebooks_vandv` module installed.

Please, see the [README] file for the requirements to run this notebook.  

[README]: https://github.com/lsst-sitcom/notebooks_vandv/blob/develop/README.md
[LVV-T2730]: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T2730

## Setup

In [169]:
test_case = "LVV-T2732"
test_exec = "LVV-EXXXX"

# az_grid = [225, 135, 45, -45]
# el_grid = [25, 35, 45, 55, 65, 75]

az_grid = [135, 45, -45, -135]
el_grid = [25, 35, 45, 55, 65, 75]

# 101 - Wide Camera
# 102 - Narrow Camera
# 103 - Fast Camera (DIMM)
camera_sal_indexes = [101, 102, 103]
exposure_times = [5., 4., 6.]  # s
base_msg = f"{test_case} {test_exec}:"

number_of_exposures = 5
n_offsets = 5
offset_size = 3.5 # degrees
track_time = 60. 
sleep_time= 1.5

### Prepare Notebook

Start with importing libraries:

In [153]:
%load_ext autoreload
%autoreload 2

import asyncio
import logging
import numpy as np
import sys
import yaml

from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.time import Time

from lsst.ts import salobj
from lsst.ts.idl.enums.MTDome import SubSystemId
from lsst.ts.idl.enums.Script import ScriptState
from lsst.ts.observatory.control import RotType
from lsst.ts.observatory.control.maintel import MTCS, ComCam
from lsst.ts.observatory.control.generic_camera import GenericCamera

from lsst.sitcom import vandv

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


---
Print out the execution info for future reference.

In [3]:
exec_info = vandv.ExecutionInfo()
print(exec_info)


Executed by isotuela on 2023-03-24T21:50:49.685.
  Running in yagan02 at summit



---
Setup the logger, the Domain, and the remote for the telescope control.

In [5]:
logging.basicConfig(format="%(asctime)s %(name)s: %(message)s", level=logging.DEBUG)
log = logging.getLogger(f"{test_case}")

In [7]:
os.environ["LSST_DDS_HISTORYSYNC"] = "200"
domain = salobj.Domain()
print(f"My user_host is {domain.user_host!r}\n")

My user_host is 'isotuela@nb-isotuela'



---
The `Script` CSC is used to record test checkpoints and allow to easy search in the EFD.

In [8]:
script = salobj.Controller("Script", index=199)
await script.start_task

script.log.info("Testing Log")

Instanciate the MTCS.

In [9]:
mtcs = MTCS(domain=domain, log=script.log)
vandv.logger.add_filter_to_mtcs()
await mtcs.start_task

[None, None, None, None, None, None, None, None, None, None]

--- 
Initialize the GenericCameras.  
You might need to modify the cell below. 

In [46]:
camera_list = []

for index in camera_sal_indexes:

    gencam = GenericCamera(domain=domain, index=index, log=script.log)
    await gencam.start_task
    #await gencam.enable()

    camera_list.append(gencam)

In [47]:
print(camera_list)

[<lsst.ts.observatory.control.generic_camera.GenericCamera object at 0x7f896ec406a0>, <lsst.ts.observatory.control.generic_camera.GenericCamera object at 0x7f896f263c10>, <lsst.ts.observatory.control.generic_camera.GenericCamera object at 0x7f896f373730>]


### Check CSC Status

In [154]:
# Ignore (set check to False) the following components:
list_of_cscs = [
    "mtaos",
    "mtdome",
    "mtdometrajectory",
    "mthexapod_1",
    "mthexapod_2",
    "mtm1m3",
    "mtm2",
    "mtrotator",
]

for csc in list_of_cscs:
    script.log.info(f"Setting {csc} check to False")
    setattr(mtcs.check, csc, False)

In [13]:
await mtcs.set_state(
    salobj.State.DISABLED, 
    components=[
        "mtmount", 
        #"mtrotator",
        #"mtptg",
    ]
)

In [14]:
await mtcs.set_state(
    salobj.State.ENABLED, 
    components=[
        "mtmount",
        # "mtrotator", # Enable this only if using the Rotator Hardware
        "mtptg",
    ]
)

In [15]:
## Increase MTMount Log Verbosity
await mtcs.rem.mtmount.cmd_setLogLevel.set_start(level=15)

In [155]:
## Comment this line if you want the CCW to follow the Rotator
await mtcs.enable_ccw_following()

In [157]:
await mtcs.stop_tracking()

In [158]:
## Home MTMount Main Axes
await mtcs.rem.mtmount.cmd_homeBothAxes.start(timeout=300)

In [159]:
await mtcs.assert_liveliness()

## Enable this only if all the MT CSCs are enabled
# await mtcs.assert_all_enabled()

In [160]:
for cam in camera_list:
    await cam.assert_liveliness()
    await cam.assert_all_enabled()

### Confirm take image each camera

In [ ]:
reason = "header_check"

await camera_list[0].take_object(exptime=1., reason=reason)
await camera_list[1].take_object(exptime=1., reason=reason)
await camera_list[2].take_object(exptime=1., reason=reason)

In [115]:
await camera_list[2].take_object(exptime=1., reason="Sync")

[2023032400077]

### Confirm Take Images in Sync

In [ ]:
res = (number_of_exposures) * len(camera_list)
for n in range(number_of_exposures):
    tasks = [asyncio.create_task(cam.take_object(exptime, reason=base_msg[:-1])) 
             for (cam, exptime) in zip(camera_list, exposure_times)]
    await asyncio.gather(*tasks)

### Other preparation

- LVV-T2713 (1.0) Establish TMA - StarTracker Axis Angle Reference

## Helper Functions

In [161]:
def generate_azel_sequence(az_seq, el_seq, el_limit=90):
    """A generator that cicles through the input azimuth and elevation sequences
    forward and backwards.
    
    Parameters
    ----------
    az_seq : `list` [`float`]
        A sequence of azimuth values to cicle through
    el_seq : `list` [`float`]
        A sequence of elevation values to cicle through
    el_limit : `float`
        Cut off limit angle in elevation to skip points when going down. 
    Yields
    ------
    `list`
        Values from the sequence.
    Notes
    -----
    This generator is designed to generate sequence of values cicling through
    the input forward and backwards. It will also reverse the list when moving
    backwards.
    Use it as follows:
    >>> az_seq = [0, 180]
    >>> el_seq = [15, 45]
    >>> seq_gen = generate_azel_sequence(az_seq, el_seq)
    >>> next(seq_gen)
    [0, 15]
    >>> next(seq_gen)
    [0, 45]
    >>> next(seq_gen)
    [180, 45]
    >>> next(seq_gen)
    [180, 15]
    >>> next(seq_gen)
    [0, 15]
    """
    i = 1
    for az in az_seq:
        for el in el_seq[::i]:
            if el > el_limit and i == -1:
                continue
            else:
                yield (az, el)
        i *= -1

In [23]:
async def wait_for_dome_in_position():
    """Wait until the dome is in position"""
    await asyncio.sleep(20)
    azMotion = await mtcs.rem.mtdome.evt_azMotion.aget()

    while not azMotion.inPosition:
        azMotion = await mtcs.rem.mtdome.evt_azMotion.aget()
        await asyncio.sleep(5.)
        
    if azMotion.state == 1.:
        await mtcs.rem.mtdome.cmd_exitFault.set_start()

In [162]:
async def take_images_in_sync(_camera_list, _exposure_times, _number_of_exposures, _reason, total_time): 
    """
    Take images in sync, which means keeping the images ID the same. 
    This will increase overhead on the camera with shorter exposure time.
    
    Parameters
    ----------
    _camera_list : list of `GenericCamera`
        A list containing the `GenericCamera` for each Camera.
    _exposure_times : list of float 
        A list containing the exposure time used on each camera.
    _reason : str 
        Reason that goes to the metadata in each image.
    _number_of_exposures : float
        Total number of exposures for each camera.
    total_time : float
        Minimum time we should spend taking images (to keep tracking in a fixed position).
    """
    assert len(_camera_list) == len(_exposure_times)

    wait_time = asyncio.create_task(asyncio.sleep(total_time))
    
    for n in range(_number_of_exposures):
        tasks = [asyncio.create_task(cam.take_object(exptime, reason=_reason)) 
                 for (cam, exptime) in zip(_camera_list, _exposure_times)]
        
        # Wait until all the tasks are complete
        await asyncio.gather(*tasks)
        
    await wait_time

## Data Acquisition

## Loop without TMA tracking and without Dome Following

The following cell is isolated to allow continuing the grid from a point where you stopped. 

In [163]:
azel_raw = [(az, el) for az, el in generate_azel_sequence(az_grid, el_grid, el_limit=90.)]

## Uncomment this to run backward azimuth grid
# azel_raw = azel_raw[::-1]

for i, (az, el) in enumerate(azel_raw):
    print(i, az, el)

0 135 25
1 135 35
2 135 45
3 135 55
4 135 65
5 135 75
6 45 75
7 45 65
8 45 55
9 45 45
10 45 35
11 45 25
12 -45 25
13 -45 35
14 -45 45
15 -45 55
16 -45 65
17 -45 75
18 -135 75
19 -135 65
20 -135 55
21 -135 45
22 -135 35
23 -135 25


In [166]:
azel

<AltAz Coordinate (obstime=60028.0404784577, location=(1819093.56876225, -5208411.6827961, -3195180.61110659) m, pressure=0.0 hPa, temperature=0.0 deg_C, relative_humidity=0.0, obswl=1.0 micron): (az, alt) in deg
    (277.92711783, 49.5285926)>

---
Uncomment and edit the lines below if you need a smaller grid for now.

In [ ]:
# az_grid_tiny = [130]
# el_grid_tiny = [40]
# azel_raw = [(az, el) for az, el in generate_azel_sequence(az_grid_tiny, el_grid_tiny)]

---
Uncomment the line bellow and replace i with the position index from the loop below

In [174]:
recover_index = 10
azel = azel_raw[recover_index::]
print(azel)

[(45, 35), (45, 25), (-45, 25), (-45, 35), (-45, 45), (-45, 55), (-45, 65), (-45, 75), (-135, 75), (-135, 65), (-135, 55), (-135, 45), (-135, 35), (-135, 25)]


---
This is the main loop. See comments in the code for details.

In [173]:
script.log.info(f"{base_msg} Serpent Walk w/ Random Offsets - Start")

# Initialize current_az to protect the Dome. 
# We only send move commands if we are going to a different az.
current_az = 0

for i, (az, el) in enumerate(azel):
    
    ## All of the following steps should be included in the point_azel command
    ## - Point the Dome
    ## - Wait the Dome to arrive    
    ## - Point the TMA
    ## - Wait the TMA to arrive
    script.log.info(f"{base_msg} Data acquisition #{i + recover_index} at az={az} and el={el} - Start")
    
    ## Disable dome following for now
    script.log.info(f"{base_msg} Dome following - Disable")
    # await mtcs.disable_dome_following()    
    mtcs.check.mtdometrajectory = False
    
    ## Start moving the Dome (only if we are going to a new Az)
    if az != current_az:
        script.log.info(f"{base_msg} Moving Dome to az={az} - Start")
        await mtcs.rem.mtdome.cmd_exitFault.set_start()
        await asyncio.sleep(5)
        await mtcs.rem.mtdome.cmd_moveAz.set_start(position=az+1.5, velocity=0)
        dome_task = asyncio.create_task(wait_for_dome_in_position())
    else:
        script.log.info(f"{base_msg} Keep Dome at az={az}")
    
    ## Point Az/El using Ra/Dec for tracking
    script.log.info(f"{base_msg} Point to az={az}, el={el} - Start")
    radec = mtcs.radec_from_azel(az, el)
    await mtcs.slew_icrs(ra=radec.ra, dec=radec.dec, rot=0, rot_type=RotType.Physical)
    script.log.info(f"{base_msg} Point to az={az}, el={el} - Done")    
    
    ## Wait until dome in position
    if az != current_az:
        await dome_task
        script.log.info(f"{base_msg} Moving Dome to az={az} - Done")

    ## Sleep and engage dome breaks
    script.log.info(f"{base_msg} Dome Brakes - Start")
    await asyncio.sleep(10)
    sub_system_ids = SubSystemId.AMCS
    await mtcs.rem.mtdome.cmd_stop.set_start(engageBrakes=True, subSystemIds=sub_system_ids)
    script.log.info(f"{base_msg} Dome Brakes - Done")
    
    exp_start = Time.now()
    script.log.info(f"{base_msg} Take image(s) original position - Start")
    await vandv.slew_and_track.take_images_in_sync_for_time(
            camera_list, 
            exposure_times,
            reason=f"{base_msg[:-1]}_{i:03d}", 
            tracktime=track_time,
            sleep=sleep_time
        )    
    
    script.log.info(f"{base_msg} Take image(s) original position - Done")
    
    script.log.info(f"{base_msg} Apply random offsets - Start")
    for j in range(n_offsets):
        random_angle = 2 * np.pi * np.random.rand()
        offset_dec = offset_size * np.cos(random_angle)
        
        sign = 1 if np.random.rand() < 0.5 else -1
        offset_ra = sign * np.sqrt(offset_size ** 2 - offset_dec ** 2) / np.cos(radec.dec.rad)
        
        script.log.info(f"{base_msg} Offset #{j} - ra = {offset_ra:.5f}, dec = {offset_dec:.5f} - Start")
        # await mtcs.offset_radec(offset_ra, offset_dec)
        await mtcs.slew_icrs(
            ra=radec.ra + offset_ra * u.deg, 
            dec=radec.dec + offset_dec * u.deg,
            rot=0, 
            rot_type=RotType.Physical
        )
        script.log.info(f"{base_msg} Offset #{j} - ra = {offset_ra:.5f}, dec = {offset_dec:.5f} - Done")
        
        script.log.info(f"{base_msg} Take image(s) - offset #{j} - Start")
        await vandv.slew_and_track.take_images_in_sync_for_time(
            camera_list, 
            exposure_times,
            reason=f"{base_msg[:-1]}_{i:03d}", 
            tracktime=track_time,
            sleep=sleep_time
        )
            
        script.log.info(f"{base_msg} Take image(s) - offset #{j} - Done")
        
        script.log.info(f"{base_msg} Slew back to original sky position - Start")
        await mtcs.slew_icrs(ra=radec.ra, dec=radec.dec, rot=0, rot_type=RotType.Physical)
        script.log.info(f"{base_msg} Slew back to original sky position - Done")
        
        script.log.info(f"{base_msg} Take image(s) original sky position - Start")
        await vandv.slew_and_track.take_images_in_sync_for_time(
            camera_list, 
            exposure_times,
            reason=f"{base_msg[:-1]}_{i:03d}", 
            tracktime=track_time,
            sleep=sleep_time
        )
            
        script.log.info(f"{base_msg} Take image(s) original sky position - Done")

    script.log.info(f"{base_msg} Apply random offsets - Done")
    
    current_az = az
    script.log.info(f"{base_msg} Take image(s) - Done")    
    script.log.info(f"{base_msg} Data acquisition #{i + recover_index} at az={az} and el={el} - Done")
    await mtcs.rem.mtdome.cmd_exitFault.set_start()
    
script.log.info(f"{base_msg} Serpent Walk Backward - Done")

CancelledError: 

## Dome Commands

The following commands allow to control the Dome. However, make sure it is in ENABLED state. Use LOVE to change its state.

### Move to position

In [39]:
await mtcs.rem.mtdome.cmd_moveAz.set_start(position=81, velocity=0)

### Stop the Dome

In [ ]:
sub_system_ids = SubSystemId.AMCS
await dome.cmd_stop.set_start(engageBrakes=True, subSystemIds=sub_system_ids)

### Recover from fault when stopping

In [164]:
await mtcs.rem.mtdome.cmd_exitFault.set_start()